In [0]:
# Download dataset
import os
os.environ['KAGGLE_USERNAME'] = "ashiishkarhade" # username from the json file
os.environ['KAGGLE_KEY'] = "b1a95537e1f638d722d282d16825c7a7" # key from the json file
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

In [0]:
import pandas as pd

FILE_PATH = 'chestxray/metadata.csv'
IMAGE_PATH = 'chestxray/images'

In [60]:
df = pd.read_csv(FILE_PATH)
print(df.shape)

(354, 28)


In [0]:
df.head()

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,infiltrate in the upper lobe of the left lung,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [0]:
import os

os.mkdir('dataset')
# COVID IMAGE FOLDER
TARGET_DIR = 'dataset/covid'
if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)

# NORMAL IMAGES FOLDER
TARGET_DIR_NORMAL = 'dataset/normal'
if not os.path.exists(TARGET_DIR_NORMAL):
    os.mkdir(TARGET_DIR_NORMAL)

In [0]:
import shutil
cnt = 0
for (i, rows) in df.iterrows():
    if rows["finding"] == "COVID-19" and rows["view"]=="PA":
        filename = rows['filename']
        image_path = os.path.join(IMAGE_PATH, filename)
        target_copy_path = os.path.join(TARGET_DIR, filename)
        shutil.copy2(image_path, target_copy_path)

In [0]:
# SAMPLING IMAGES FROM KAGGLE DATASet
import random
KAGGLE_PATH = '/content/chest_xray/train/NORMAL'

In [0]:
image_names = os.listdir(KAGGLE_PATH)
random.shuffle(image_names)

In [0]:
for i in range(142):
    image_name = image_names[i]
    img_path = os.path.join(KAGGLE_PATH, image_name)
    targ_path = os.path.join(TARGET_DIR_NORMAL, image_name)
    shutil.copy2(img_path, targ_path)

In [0]:
# TRAIN_TEST_SPLIT INTO TWO FOLDERS
# TRAIN -> COVID, NORMAL
# VAL -> COVID, NORMAL
os.mkdir('dataset/train')
os.mkdir('dataset/val')
os.mkdir('dataset/train/normal')
os.mkdir('dataset/val/normal')
os.mkdir('dataset/train/covid')
os.mkdir('dataset/val/covid')

train_dir = 'dataset/train'
val_dir = 'dataset/val'

In [0]:
from PIL import Image
normals = os.listdir('dataset/normal')
covs = os.listdir('dataset/covid')
TRAINING_SPLIT = 112

train_normal = normals[:TRAINING_SPLIT]
test_normal = normals[TRAINING_SPLIT:]
train_covid = covs[:TRAINING_SPLIT]
test_covid = covs[TRAINING_SPLIT:]

In [0]:
for filename in train_normal:
    img_pth = os.path.join('dataset/normal', filename)
    target_pth = os.path.join('dataset/train/normal', filename)
    shutil.copy2(img_pth, target_pth)

for filename in test_normal:
    img_pth = os.path.join('dataset/normal', filename)
    target_pth = os.path.join('dataset/val/normal', filename)
    shutil.copy2(img_pth, target_pth)

for filename in train_covid:
    img_pth = os.path.join('dataset/covid', filename)
    target_pth = os.path.join('dataset/train/covid', filename)
    shutil.copy2(img_pth, target_pth)

for filename in test_covid:
    img_pth = os.path.join('dataset/covid', filename)
    target_pth = os.path.join('dataset/val/covid', filename)
    shutil.copy2(img_pth, target_pth)

In [0]:
# deleting every other folders 
shutil.rmtree('chest_xray')
shutil.rmtree('chestxray')
shutil.rmtree('dataset/covid')
shutil.rmtree('dataset/normal')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [93]:
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size = (224,224),
    batch_size=32,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/val',
    target_size = (224,224),
    batch_size=32,
    class_mode = 'binary'
)

Found 224 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [94]:
train_generator.class_indices

{'covid': 0, 'normal': 1}

In [0]:
# CNN BASED MODEL

model = Sequential([
                    # BLOCK 1
                    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (224, 224, 3)),
                    Conv2D(64, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 2
                    Conv2D(128, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 3
                    Conv2D(128, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 4
                    Conv2D(264, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # block 5
                    Flatten(),
                    Dense(64, activation='relu'),
                    Dropout(0.5),
                    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [117]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 54, 54, 128)      

In [118]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = test_generator,
    validation_steps = 2
)

Epoch 1/10
8/8 [==============================] - 10s 1s/step - loss: 1.0406 - accuracy: 0.5469 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 10s 1s/step - loss: 0.6866 - accuracy: 0.5664 - val_loss: 0.6893 - val_accuracy: 0.5500
Epoch 3/10
8/8 [==============================] - 10s 1s/step - loss: 0.6566 - accuracy: 0.6523 - val_loss: 0.6237 - val_accuracy: 0.6500
Epoch 4/10
8/8 [==============================] - 10s 1s/step - loss: 0.5309 - accuracy: 0.7188 - val_loss: 0.4123 - val_accuracy: 0.9333
Epoch 5/10
8/8 [==============================] - 10s 1s/step - loss: 0.3692 - accuracy: 0.8711 - val_loss: 0.5088 - val_accuracy: 0.9167
Epoch 6/10
8/8 [==============================] - 10s 1s/step - loss: 0.2783 - accuracy: 0.9023 - val_loss: 0.2352 - val_accuracy: 0.9500
Epoch 7/10
8/8 [==============================] - 10s 1s/step - loss: 0.2543 - accuracy: 0.9102 - val_loss: 0.2571 - val_accuracy: 0.9000
Epoch 8/10
8/8 [==================